Installing Wikipedia module for extracting information to populate the map

In [1]:
#pip install wikipedia

In [2]:
#pip install requests

In [3]:
#pip install beatifulsoup

In [4]:
#response = requests.get(
#	url="https://en.wikipedia.org/wiki/Algarve_International_Circuit",
#)
#soup = soup(response.content, 'html.parser')
#img = soup.find('infobox vcard')
#img

In [5]:
#print(img)

In [6]:
import folium
import pandas as pd
import pyodbc

In [7]:
#from bs4 import BeautifulSoup as soup
#import requests

In [8]:
import wikipedia as wiki

In [9]:
server = 'f1server.database.windows.net'
database = 'WackyRacesF1'
username = 'PDS'
password = '{Formula1}'   
driver= '{ODBC Driver 17 for SQL Server}'

#with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
#    with conn.cursor() as cursor:
#        cursor.execute("SELECT * FROM dbo.circuits")
#        row = cursor.fetchone()
#        while row:
#            print (str(row[0]) + " " + str(row[1]))
#            row = cursor.fetchone()

In [10]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
circuits_query = "SELECT DISTINCT c.circuitId, c.name, c.location, c.country, lat, lng, alt, c.url FROM circuits c LEFT JOIN races r ON r.circuitId = c.circuitId WHERE year >= 2015 and year <= 2022;"
circuits = pd.read_sql(circuits_query, cnxn)

In [11]:
circuits

,circuitId,name,location,country,lat,lng,alt,url
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-378.497009,144.968002,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,Sepang International Circuit,Kuala Lumpur,Malaysia,276.083008,101.737999,18.0,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,Bahrain International Circuit,Sakhir,Bahrain,260.325012,505.105988,7.0,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.570000,226.110992,109.0,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,Istanbul Park,Istanbul,Turkey,409.516998,29.405001,130.0,http://en.wikipedia.org/wiki/Istanbul_Park
5,6,Circuit de Monaco,Monte-Carlo,Monaco,437.346985,742.056030,7.0,http://en.wikipedia.org/wiki/Circuit_de_Monaco
6,7,Circuit Gilles Villeneuve,Montreal,Canada,45.500000,-735.228027,13.0,http://en.wikipedia.org/wiki/Circuit_Gilles_Vi...
7,9,Silverstone Circuit,Silverstone,UK,520.786011,-101.694000,153.0,http://en.wikipedia.org/wiki/Silverstone_Circuit
8,10,Hockenheimring,Hockenheim,Germany,493.278015,856.583008,103.0,http://en.wikipedia.org/wiki/Hockenheimring
9,11,Hungaroring,Budapest,Hungary,475.789001,192.485992,264.0,http://en.wikipedia.org/wiki/Hungaroring


In [16]:
details = []

for name in circuits['name']:
    detail = wiki.WikipediaPage(title=name).summary
    details.append(detail)

In [17]:
circuits['details'] = details

In [53]:
def generate_map(df):
    
    my_map = folium.Map(location=[38.72763592384391, -9.138247239298348],
                    zoom_start=2,
                    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                    attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
                   )
    
    for index, row in df.iterrows():
        
        html=f"""
        <h1><img src='data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz4KPCEtLSBHZW5lcmF0b3I6IEFkb2JlIElsbHVzdHJhdG9yIDI1LjIuMywgU1ZHIEV4cG9ydCBQbHVnLUluIC4gU1ZHIFZlcnNpb246IDYuMDAgQnVpbGQgMCkgIC0tPgo8c3ZnIHZlcnNpb249IjEuMSIgaWQ9IkxheWVyXzEiIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIgeG1sbnM6eGxpbms9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkveGxpbmsiIHg9IjBweCIgeT0iMHB4IgoJIHZpZXdCb3g9IjAgMCAxNTQ2LjIgOTcwLjIiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDE1NDYuMiA5NzAuMjsiIHhtbDpzcGFjZT0icHJlc2VydmUiPgo8c3R5bGUgdHlwZT0idGV4dC9jc3MiPgoJLnN0MHtmaWxsOm5vbmU7c3Ryb2tlOiMzNzQxNTE7c3Ryb2tlLXdpZHRoOjcuODU0NTtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0MXtmaWxsOm5vbmU7c3Ryb2tlOiNFRjQ0NDQ7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0MntmaWxsOm5vbmU7c3Ryb2tlOiMzQjgyRjY7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQoJLnN0M3tmaWxsOm5vbmU7c3Ryb2tlOiNGQkJGMjQ7c3Ryb2tlLXdpZHRoOjEuOTYzNjtzdHJva2UtbWl0ZXJsaW1pdDozLjkyNzM7fQo8L3N0eWxlPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDAiIGQ9Ik01NTMuOCwyMjkuOGMzMy4xLDY2LjUsNjIuNiwxMzUuMiw5NC4zLDIwMi4zCgljMTAuNCwyMi4xLDIwLjMsNDUuNSwzMi40LDY2LjhjOC40LDE4LjYsMTguNCwzOSwzMy40LDUzYzEyLjcsMTUuMywzNi4yLDE5LjIsNTUsMTUuN2MxNy0zLjUsMzIuNC0xMi44LDQ3LjEtMjEuNgoJYzIyLjYtMTQuOCw0Ni40LTI4LjgsNjguNy00NC4yYzEwOS42LTc0LjMsMjguMy0xMzMsODMuNS0yMThjMjMuOS0zOC42LDY4LjktNjMsMTEzLjktNjUuOGMyMC44LTMuMSw0My0wLjcsNjMuOC0yLjkKCWMyNS4yLTEuOSw1MC40LTcuOCw3NC42LTE0LjdjNDAuNy0xMi45LDgwLjEtMzEuOCwxMTguOC00OS4xYzIxLjYtMTIuOCw1MC41LTE4LjIsNjEuOC00Mi4yYzExLjMtMjYuOC0wLjEtNTEuMSwzNy4zLTYwLjkKCWMxMy45LTMsMjcuNCwyLjYsNDAuMyw1LjljMTMuMSwzLjMsMjYuMyw0LjQsMzkuMywzLjljMTguMy00LjMsMjctMjUuMiwxMC44LTM5LjNjLTExLjUtOC42LTI2LjktOS45LTQwLjMtOS44CgljLTIzMiw3LjQtNDY1LjIsNC4zLTY5Ny4xLDcuOWMtMTQyLjYsMC43LTI4NSwxLjMtNDI3LjEsMWMtMTgsMC42LTMxLDE0LjctMjguNSwzMi40YzguMywxNi4xLDE2LjEsMzUuMiwzLjksNTEuMQoJYy01LjQsMTAuOC0xOC4zLDE2LjItMjcuNSwyMi42Yy0xNy42LDExLjctMzcuMywyMC42LTUzLDM0LjRjLTI1LjcsMTguNS00My43LDQ1LjItNjAuOSw3MC43Yy0yOC4yLDQyLjMtNDguOSw4OC43LTY2LjgsMTM1LjUKCUM3Ni44LDUwMy41LDMwLjQsNjQ3LjYsMTcuOCw3OTYuNGMtMy40LDM0LTguOSw2Ny4xLTguOCwxMDAuMWM1LjQsMzMuOSwyNS42LDMzLjksNTQsMzUuM2MyOC41LDguNywzMiwyOC45LDYzLjgsMjkuNAoJYzE5LjUtMC40LDM4LjctNy41LDU2LTEzLjhjMTkuMS02LjUsMzguNy0xOC4xLDQzLjItMzguM2M1LjUtMzIuNC0yMC4yLTY0LjctMzYuMy05MC4zYy0xNy4xLTI4LjUtMzQuMy01Ny4xLTUwLjEtODUuNAoJYy01MS4xLTg0LjQtMjYtMjA2LjIsNTktMjU3LjJjMjAuOC0xNi4xLDU1LjUtMzcuMiw3NC42LTkuOGM3LjYsMTIuNSw5LjMsMjcuMiwxMy43LDQwLjNjNS41LDIzLjksMjguMSw0MS44LDUyLjEsMzkuMgoJYzE5LjctMi4xLDM5LjMtOS42LDU2LTE4LjdjMzYuMy0xOC42LDMxLjEtNjUuMSwxNC43LTk1LjNjLTE4LjMtNDAuMi00Ni43LTc0LjktNjYuOC0xMTIuOWMtMTcuOC0yOC4yLTM3LjItNjUtMTkuNi05Ni4yCgljNS4yLTE1LjcsMTcuOC0yNy41LDI4LjUtMzguM2M5LjMtMTEsMTktMjIsMjguNS0zMS40YzI0LjUtMjguMiw1NS45LTcxLDk3LjItNTMuOWMyNy41LDEzLjksMzkuNCw0NS41LDUwLjEsNzEuNwoJQzUzNS4yLDE5MS41LDU0NS4xLDIxMC4yLDU1My44LDIyOS44eiIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDEiIGQ9Ik01NTMuOCwyMjkuOGMzNi44LDc0LjQsNjkuOSwxNTAuOCwxMDUuMSwyMjUuOAoJYzMzLjQsNjYuMiw1NywxNTAuMywxNDcuMyw5Ni4yYzUxLTM1LjEsMTIxLTYxLDEzOC40LTEyNi43YzYtMjcuNSwwLjMtNTUuOCwyLTgzLjVjNS45LTQ0LjcsMzQuMi04NC41LDczLjYtMTA2CgljNDMuMS0yNS40LDk1LTE1LjgsMTQyLjQtMjIuNmM1Ny4yLTcuNCwxMTEuMS0zMS42LDE2My01NiIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDIiIGQ9Ik0xMzI1LjUsMTU3LjJjMTMuOC01LjcsMjgtMTIuOSw0MS4yLTE5LjZjMTMuOC01LjcsMjguMS0xNC4zLDM0LjQtMjguNQoJYzctMTIuOSw0LTI5LjEsMTAuOC00Mi4yYzcuMS0xNCwyNi4xLTIxLjksNDEuMi0xOC43YzkuOS0wLjQsMTkuNyw1LjQsMjkuNSw2LjljMTMsMS45LDI3LjYsNS44LDQwLjMsMQoJYzI1LjgtMTEuNiwxNC41LTM5LjUtMTAuOC00NS4yYy0xNi40LTMuMy0zMy41LTEuNi01MC4xLTFjLTIxLjIsMC4yLTQyLjcsMS02My44LDJjLTMzOS45LDIuOC02ODAsNi4zLTEwMjAuMSw1LjkKCWMtMjYuNS0yLjktNTEsMTAuOC0zOS4zLDQwLjNjMjIuMiwzNC4yLTcuOSw1Ny0zNi4zLDcxLjdjLTM0LDE5LjQtNjYuOSw0My04OC40LDc2LjZjLTQyLjUsNTctNzAuMywxMjMtOTQuMywxODkuNQoJQzcwLjYsNTIxLjMsMzEuOCw2NTIsMTguNyw3ODYuNSIvPgo8cGF0aCB2ZWN0b3ItZWZmZWN0PSJub24tc2NhbGluZy1zdHJva2UiIGNsYXNzPSJzdDMiIGQ9Ik0xOC43LDc4Ni41Yy0yLjksMjMuNi00LjgsNDguMi03LjksNzEuN2MtMS41LDIyLTcuNiw0OS41LDExLjgsNjUuOAoJYzEwLDcuMSwyMi42LDcuMSwzNC40LDYuOWM3LjksMC44LDE1LjQsNSwyMi42LDcuOWMyNy42LDI1LjksNTAuNCwyNy40LDg1LjQsMTQuN2M0MC43LTEwLjUsNzYuOC0zMi44LDU2LTc5LjUKCWMtMTAuMy0yNi45LTI5LjItNDkuNy00My4yLTc0LjZjLTIwLjMtMzYuNC00Ni4zLTcxLjQtNTcuOS0xMTEuOWMtMTIuNS00NC40LTYuNS05My4xLDEwLjgtMTM1LjVjMjAuMi00MS45LDU3LjMtNzMuNSw5OC4yLTk0LjMKCWMyMi4yLTExLjgsNDcuMSwwLjQsNTEuMSwyNS41YzcuOCwzMCwxNiw2MS4yLDUzLDYyLjhjMTQuNiwwLjksMjguOS00LjgsNDIuMi05LjhjMjQtNi41LDQ2LjUtMjUuOSw0Ny4xLTUyCgljMS4yLTMzLjUtMTcuNS02My42LTM0LjQtOTEuM2MtMTcuNS0yOC4zLTM2LjEtNTYuMi01Mi04NS40Yy0xOS43LTMxLTI4LjUtNjYuNC01LjktOTguMmMxOC4yLTIzLDM5LjItNDQuMSw1OC45LTY1LjgKCWMxMi44LTE2LDI3LjYtMzEuNiw0NS4yLTQyLjJjMTkuNi05LjIsNDYuNC03LjcsNTkuOSwxMC44YzE2LjYsMTUuOCwyNSwzOC4xLDMzLjQsNTguOWM3LjYsMTkuOSwxNy44LDM5LjQsMjYuNSw1OC45Ii8+Cjwvc3ZnPgo='/></p>
        </h1>
        <h1><p> {row['name']}</p></h1>
        <p><p>Circuit information:</p>
        <ul>
            <li>Country name: {row['country']}</li>
            <li>City name: {row['location']}</li>
        <details>
            <summary>More details</summary>
            {row['details']}
        </details>     
        </ul>
        </p>
        <p>To know more: <a href={row['url']}>link</a></p>
        """
        
        iframe = folium.IFrame(html=html, width=300, height=300)
        
        popup = folium.Popup(iframe, max_width=2650)
        
        folium.Marker(location=[row['lat'],row['lng']],popup=popup,tooltip=row['name'],radius=3, color='Black').add_to(my_map)

        
    return my_map

In [54]:
generate_map(circuits)

In [55]:
details=[]

for name in circuits['name']:
    try:
        detail = wiki.WikipediaPage(title=name).coordinates
        lat, long = detail
        details.append((lat,long))
    except:
        print(name)

Nürburgring
Miami International Autodrome


In [47]:
details

[(Decimal('-37.84972221999999675290382583625614643096923828125'),
  Decimal('144.96833333000000720858224667608737945556640625')),
 (Decimal('2.760555559999999797327063788543455302715301513671875'),
  Decimal('101.7374999999999971578290569595992565155029296875')),
 (Decimal('26.032499999999998863131622783839702606201171875'),
  Decimal('50.51055556000000024141627363860607147216796875')),
 (Decimal('41.57000000000000028421709430404007434844970703125'),
  Decimal('2.2611111099999998685916580143384635448455810546875')),
 (Decimal('40.95166667000000160214767674915492534637451171875'),
  Decimal('29.405000000000001136868377216160297393798828125')),
 (Decimal('43.73472222000000186881152330897748470306396484375'),
  Decimal('7.420555560000000383524820790626108646392822265625')),
 (Decimal('45.5005777800000004162939148955047130584716796875'),
  Decimal('-73.522461109999994732788763940334320068359375')),
 (Decimal('52.0786111099999970974749885499477386474609375'),
  Decimal('-1.01694444000000006

In [34]:
lat, long = details[0]

-37.84972221999999675290382583625614643096923828125 144.96833333000000720858224667608737945556640625


In [35]:
type(lat)

decimal.Decimal

In [ ]:
print(lat,long)